In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [3]:
# read in with pandas
winter1 = pd.read_csv ('mht20-21data.txt')
winter2 = pd.read_csv ('mht21-22data.txt')
winter3 = pd.read_csv ('mht22-23data.txt')

In [4]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [5]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
mht = LocationInfo(
    'Manchester', 'New Hampshire', 
    latitude=42.9956,  
    longitude=-71.4548,
    timezone='America/New_Hampshire'
)

mht

LocationInfo(name='Manchester', region='New Hampshire', timezone='America/New_Hampshire', latitude=42.9956, longitude=-71.4548)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(mht.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [24]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(mht.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [26]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(mht.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [28]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:58:08  2020-12-01 16:11:48  2020-12-01
1   2020-12-02 06:59:11  2020-12-02 16:11:31  2020-12-02
2   2020-12-03 07:00:14  2020-12-03 16:11:17  2020-12-03
3   2020-12-04 07:01:15  2020-12-04 16:11:06  2020-12-04
4   2020-12-05 07:02:15  2020-12-05 16:10:57  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:29:37  2021-02-24 17:28:55  2021-02-24
86  2021-02-25 06:28:02  2021-02-25 17:30:11  2021-02-25
87  2021-02-26 06:26:26  2021-02-26 17:31:27  2021-02-26
88  2021-02-27 06:24:49  2021-02-27 17:32:42  2021-02-27
89  2021-02-28 06:23:12  2021-02-28 17:33:58  2021-02-28

[90 rows x 3 columns]


In [30]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:57:52  2021-12-01 16:11:52  2021-12-01
1   2021-12-02 06:58:56  2021-12-02 16:11:35  2021-12-02
2   2021-12-03 06:59:59  2021-12-03 16:11:20  2021-12-03
3   2021-12-04 07:01:01  2021-12-04 16:11:08  2021-12-04
4   2021-12-05 07:02:01  2021-12-05 16:10:58  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:30:00  2022-02-24 17:28:36  2022-02-24
86  2022-02-25 06:28:25  2022-02-25 17:29:52  2022-02-25
87  2022-02-26 06:26:49  2022-02-26 17:31:08  2022-02-26
88  2022-02-27 06:25:13  2022-02-27 17:32:24  2022-02-27
89  2022-02-28 06:23:35  2022-02-28 17:33:40  2022-02-28

[90 rows x 3 columns]


In [32]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:57:36  2022-12-01 16:11:56  2022-12-01
1   2022-12-02 06:58:41  2022-12-02 16:11:39  2022-12-02
2   2022-12-03 06:59:44  2022-12-03 16:11:23  2022-12-03
3   2022-12-04 07:00:46  2022-12-04 16:11:11  2022-12-04
4   2022-12-05 07:01:46  2022-12-05 16:11:00  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:30:23  2023-02-24 17:28:18  2023-02-24
86  2023-02-25 06:28:48  2023-02-25 17:29:34  2023-02-25
87  2023-02-26 06:27:13  2023-02-26 17:30:50  2023-02-26
88  2023-02-27 06:25:36  2023-02-27 17:32:06  2023-02-27
89  2023-02-28 06:23:59  2023-02-28 17:33:21  2023-02-28

[90 rows x 3 columns]


In [34]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [36]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MHT,2020-12-01 00:53:00,0.76,-RA BR,2020-12-01,2020-12-01 06:58:08,2020-12-01 16:11:48
1,MHT,2020-12-01 01:53:00,2.54,RA,2020-12-01,2020-12-01 06:58:08,2020-12-01 16:11:48
2,MHT,2020-12-01 02:53:00,4.32,RA BR,2020-12-01,2020-12-01 06:58:08,2020-12-01 16:11:48
3,MHT,2020-12-01 03:53:00,1.02,-RA BR,2020-12-01,2020-12-01 06:58:08,2020-12-01 16:11:48
4,MHT,2020-12-01 04:53:00,1.27,-RA BR,2020-12-01,2020-12-01 06:58:08,2020-12-01 16:11:48
...,...,...,...,...,...,...,...
236,MHT,2021-02-27 15:53:00,0.51,-SN BR,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42
237,MHT,2021-02-27 16:53:00,0.25,-RA,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42
238,MHT,2021-02-27 17:53:00,1.52,-RA BR,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42
239,MHT,2021-02-27 18:53:00,1.02,-RA FG,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42


In [38]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MHT,2021-12-02 09:53:00,0.0001,-RA,2021-12-02,2021-12-02 06:58:56,2021-12-02 16:11:35
1,MHT,2021-12-02 10:53:00,0.5100,-RA,2021-12-02,2021-12-02 06:58:56,2021-12-02 16:11:35
2,MHT,2021-12-02 11:53:00,1.2700,-RA,2021-12-02,2021-12-02 06:58:56,2021-12-02 16:11:35
3,MHT,2021-12-02 12:53:00,0.5100,-RA,2021-12-02,2021-12-02 06:58:56,2021-12-02 16:11:35
4,MHT,2021-12-02 13:53:00,0.5100,-RA,2021-12-02,2021-12-02 06:58:56,2021-12-02 16:11:35
...,...,...,...,...,...,...,...
264,MHT,2022-02-26 00:53:00,0.0001,-SN,2022-02-26,2022-02-26 06:26:49,2022-02-26 17:31:08
265,MHT,2022-02-26 01:53:00,0.0001,-SN,2022-02-26,2022-02-26 06:26:49,2022-02-26 17:31:08
266,MHT,2022-02-26 02:53:00,0.0001,-SN,2022-02-26,2022-02-26 06:26:49,2022-02-26 17:31:08
267,MHT,2022-02-26 05:53:00,0.0001,-SN,2022-02-26,2022-02-26 06:26:49,2022-02-26 17:31:08


In [40]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MHT,2022-12-01 00:53:00,7.8700,-RA,2022-12-01,2022-12-01 06:57:36,2022-12-01 16:11:56
1,MHT,2022-12-03 14:53:00,0.0001,-RA,2022-12-03,2022-12-03 06:59:44,2022-12-03 16:11:23
2,MHT,2022-12-03 15:53:00,0.0001,-RA,2022-12-03,2022-12-03 06:59:44,2022-12-03 16:11:23
3,MHT,2022-12-03 16:53:00,0.0001,-RA,2022-12-03,2022-12-03 06:59:44,2022-12-03 16:11:23
4,MHT,2022-12-03 17:53:00,0.0001,-RA,2022-12-03,2022-12-03 06:59:44,2022-12-03 16:11:23
...,...,...,...,...,...,...,...
318,MHT,2023-02-26 15:53:00,0.2500,-SN,2023-02-26,2023-02-26 06:27:13,2023-02-26 17:30:50
319,MHT,2023-02-26 17:53:00,0.0001,-SN BR,2023-02-26,2023-02-26 06:27:13,2023-02-26 17:30:50
320,MHT,2023-02-26 18:53:00,0.0001,-SN,2023-02-26,2023-02-26 06:27:13,2023-02-26 17:30:50
321,MHT,2023-02-26 19:53:00,0.0001,-SN,2023-02-26,2023-02-26 06:27:13,2023-02-26 17:30:50


In [42]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
6,MHT,2020-12-05 10:53:00,0.0001,-RA,2020-12-05,2020-12-05 07:02:15,2020-12-05 16:10:57
7,MHT,2020-12-05 11:53:00,1.2700,-RA BR,2020-12-05,2020-12-05 07:02:15,2020-12-05 16:10:57
8,MHT,2020-12-05 12:53:00,2.0300,-RA BR,2020-12-05,2020-12-05 07:02:15,2020-12-05 16:10:57
9,MHT,2020-12-05 13:53:00,2.5400,RA BR,2020-12-05,2020-12-05 07:02:15,2020-12-05 16:10:57
10,MHT,2020-12-05 14:53:00,4.3200,RA BR,2020-12-05,2020-12-05 07:02:15,2020-12-05 16:10:57
...,...,...,...,...,...,...,...
233,MHT,2021-02-27 12:53:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42
234,MHT,2021-02-27 13:53:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42
235,MHT,2021-02-27 14:53:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42
236,MHT,2021-02-27 15:53:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42


In [44]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [46]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
6,MHT,2020-12-05 10:53:00,0.0001,-RA,2020-12-05,2020-12-05 07:02:15,2020-12-05 16:10:57,2020-12-05 08:02:15,2020-12-05 15:10:57
7,MHT,2020-12-05 11:53:00,1.2700,-RA BR,2020-12-05,2020-12-05 07:02:15,2020-12-05 16:10:57,2020-12-05 08:02:15,2020-12-05 15:10:57
8,MHT,2020-12-05 12:53:00,2.0300,-RA BR,2020-12-05,2020-12-05 07:02:15,2020-12-05 16:10:57,2020-12-05 08:02:15,2020-12-05 15:10:57
9,MHT,2020-12-05 13:53:00,2.5400,RA BR,2020-12-05,2020-12-05 07:02:15,2020-12-05 16:10:57,2020-12-05 08:02:15,2020-12-05 15:10:57
10,MHT,2020-12-05 14:53:00,4.3200,RA BR,2020-12-05,2020-12-05 07:02:15,2020-12-05 16:10:57,2020-12-05 08:02:15,2020-12-05 15:10:57
...,...,...,...,...,...,...,...,...,...
232,MHT,2021-02-27 11:53:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42,2021-02-27 07:24:49,2021-02-27 16:32:42
233,MHT,2021-02-27 12:53:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42,2021-02-27 07:24:49,2021-02-27 16:32:42
234,MHT,2021-02-27 13:53:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42,2021-02-27 07:24:49,2021-02-27 16:32:42
235,MHT,2021-02-27 14:53:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42,2021-02-27 07:24:49,2021-02-27 16:32:42


In [48]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [50]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [54]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 1.2531141025641028
2021-2022: 0.976175
2022-2023: 0.8674471910112359


In [56]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MHT,2020-12-01 00:53:00,0.76,-RA BR,2020-12-01,2020-12-01 06:58:08,2020-12-01 16:11:48
1,MHT,2020-12-01 01:53:00,2.54,RA,2020-12-01,2020-12-01 06:58:08,2020-12-01 16:11:48
2,MHT,2020-12-01 02:53:00,4.32,RA BR,2020-12-01,2020-12-01 06:58:08,2020-12-01 16:11:48
3,MHT,2020-12-01 03:53:00,1.02,-RA BR,2020-12-01,2020-12-01 06:58:08,2020-12-01 16:11:48
4,MHT,2020-12-01 04:53:00,1.27,-RA BR,2020-12-01,2020-12-01 06:58:08,2020-12-01 16:11:48
...,...,...,...,...,...,...,...
230,MHT,2021-02-22 23:53:00,2.29,-SN BR,2021-02-22,2021-02-22 06:32:45,2021-02-22 17:26:21
231,MHT,2021-02-23 00:53:00,0.51,-RA BR,2021-02-23,2021-02-23 06:31:12,2021-02-23 17:27:38
238,MHT,2021-02-27 17:53:00,1.52,-RA BR,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42
239,MHT,2021-02-27 18:53:00,1.02,-RA FG,2021-02-27,2021-02-27 06:24:49,2021-02-27 17:32:42


In [58]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_21396\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [60]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [61]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [64]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.8635399999999999
2021-2022: 0.5524877192982456
2022-2023: 0.3381075268817204
